# DNN Tensorflow

In [1]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn import svm
import pylab as pl
import numpy as np
import pandas as pd  
from sklearn.model_selection import train_test_split

In [9]:
#read the data
df = pd.read_csv('./data/joinedData.csv', sep=r'\s*(?:\||\#|\,)\s*',
                 engine='python')

#change the classes to numbers
Class = {'LUAD': 0,'BRCA': 1,'KIRC': 2,'PRAD': 3,'COAD': 4} 
df.Class = [Class[item] for item in df.Class] 
df = df.drop('Unnamed: 0',1)
df = df.drop('Unnamed: 0.1',1)
df.head()


,Class,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,3,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,0,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,3,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,1,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [10]:
df = df[['Class','gene_219', 'gene_220', 'gene_450', 'gene_1858', 'gene_3439',
       'gene_3737', 'gene_3921', 'gene_6733', 'gene_7421', 'gene_7896',
       'gene_7964', 'gene_9175', 'gene_9176', 'gene_13818', 'gene_14114',
       'gene_15895', 'gene_15898', 'gene_16169', 'gene_16392', 'gene_18135']]

In [11]:
#Split the dataframe into X and y, y containing only the class column
X = df.drop('Class', axis=1).values
y = df['Class'].values
y = np.asarray(y)
X

#Standardize data
X = (X - X.mean()) / (X.max() - X.min())

In [12]:
#Split the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  
X_train

array([[-0.15995426, -0.15995426, -0.13509847, ..., -0.15995426,
        -0.15995426, -0.15995426],
       [-0.14219518, -0.15995426, -0.15995426, ..., -0.12803426,
        -0.09737733, -0.12803426],
       [-0.11523739, -0.15995426, -0.15995426, ..., -0.14169652,
        -0.11523739, -0.08104321],
       ...,
       [-0.15995426, -0.15995426, -0.15995426, ..., -0.1160455 ,
        -0.06992626, -0.02260892],
       [ 0.09685067, -0.03547744, -0.15995426, ...,  0.03838947,
        -0.01464724,  0.34153326],
       [-0.15995426, -0.15995426, -0.15995426, ..., -0.15995426,
        -0.1307714 , -0.15995426]])

In [13]:
#get the estimator
estimator = tf.estimator.DNNClassifier(
    feature_columns=[tf.feature_column.numeric_column('x', shape=X_train.shape[1:])],
    hidden_units=[1000, 500, 250], 
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l1_regularization_strength=0.001
    ), #optimizer was used to improve the estimator
    n_classes=5) #the number of label classes, we have 5

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\erika\\AppData\\Local\\Temp\\tmpq0s3yild', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B703F83A90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
# defining the training inputs
train = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=X_test.shape[0],
    num_epochs=None,
    shuffle=False,
    num_threads=1
    ) 

In [15]:
estimator.train(input_fn = train,steps=1000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\erika\AppData\Local\Temp\tmpq0s3yild\model.ckpt.
INFO:tensorflow:loss = 259.85513, step = 1
INFO:tensorflow:global_step/sec: 56.0467
INFO:tensorflow:loss = 0.23800407, step = 101 (1.786 sec)
INFO:tensorflow:global_step/sec: 60.2932
INFO:tensorflow:loss = 0.1217288, step = 201 (1.659 sec)
INFO:tensorflow:global_step/sec: 76.4235
INFO:tens

In [16]:
# defining the test inputs
input_fn2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test, 
    shuffle=False,
    batch_size=X_test.shape[0],
    num_epochs=None)

In [17]:
#evaluate the estimator
estimator.evaluate(input_fn2,steps=1000) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-11T14:04:50Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpq0s3yild\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2019-04-11-14:04:57
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.99378884, average_loss = 0.011342852, global_step = 1

{'accuracy': 0.99378884,
 'average_loss': 0.011342852,
 'loss': 1.8261992,
 'global_step': 1000}

In [18]:
#evaluate the estimator
estimator.evaluate(input_fn2,steps=1000) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-11T14:04:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpq0s3yild\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2019-04-11-14:05:05
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.99378884, average_loss = 0.011342852, global_step = 1000, loss = 1.8261992
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 

{'accuracy': 0.99378884,
 'average_loss': 0.011342852,
 'loss': 1.8261992,
 'global_step': 1000}

In [20]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    shuffle=False)

In [21]:
#predict the results
pred_results = estimator.predict(input_fn=pred_input_fn)

In [22]:
#print the results, example of the result is array([b'2'], dtype=object) 
#b = 2 = KIRC
for i in pred_results:
    print (i)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpq0s3yild\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-2.2971163,  7.8048744, -4.2888746, -2.9452417, -2.8346148],
      dtype=float32), 'probabilities': array([4.0994088e-05, 9.9990797e-01, 5.5938594e-06, 2.1440939e-05,
       2.3949066e-05], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-3.8304253, -3.5143056,  7.772824 , -2.8709671, -2.3352294],
      dtype=float32), 'probabilities': array([9.1355705e-06, 1.2532119e-05, 9.9991369e-01, 2.3846394e-05,
       4.0746545e-05], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object)}
{'logits': array([ 8.386995 , -3.606881 , -3.208501 , -3.5293546, -2.7608728],
      dtype=float32)

       6.8945669e-06], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-2.2159019,  8.639974 , -4.7355003, -3.4544418, -3.2766733],
      dtype=float32), 'probabilities': array([1.9290301e-05, 9.9996686e-01, 1.5527125e-06, 5.5904611e-06,
       6.6780794e-06], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-2.8142276,  9.379574 , -5.0591087, -3.5951498, -3.3598328],
      dtype=float32), 'probabilities': array([5.0616777e-06, 9.9998927e-01, 5.3623506e-07, 2.3181653e-06,
       2.9331979e-06], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-2.902073 ,  8.848392 , -4.7296696, -3.3173544, -3.030091 ],
      dtype=float32), 'probabilities': array([7.8854937e-06, 9.9997866e-01, 1.2679839e-06, 5.2056444e-06,
       6.9379503e-06], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array(

       1.2429674e-06], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-4.2081566, -3.1101778, -3.2158625,  6.97106  , -2.777489 ],
      dtype=float32), 'probabilities': array([1.3959249e-05, 4.1851217e-05, 3.7653903e-05, 9.9984813e-01,
       5.8370475e-05], dtype=float32), 'class_ids': array([3], dtype=int64), 'classes': array([b'3'], dtype=object)}
{'logits': array([-4.207425 , -3.8037555,  8.589644 , -3.1228607, -2.69978  ],
      dtype=float32), 'probabilities': array([2.7687981e-06, 4.1457461e-06, 9.9997234e-01, 8.1905218e-06,
       1.2504118e-05], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object)}
{'logits': array([-2.5859156,  8.103341 , -4.3778057, -3.0462158, -2.784338 ],
      dtype=float32), 'probabilities': array([2.27870914e-05, 9.99940276e-01, 3.79735206e-06, 1.43807965e-05,
       1.86859452e-05], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': a

In [23]:
y_test

array([1, 2, 0, 0, 0, 0, 1, 2, 1, 2, 3, 2, 2, 3, 1, 1, 0, 0, 3, 1, 2, 1,
       3, 0, 4, 1, 4, 1, 2, 1, 2, 0, 1, 1, 0, 4, 1, 3, 0, 1, 4, 1, 1, 2,
       0, 0, 3, 4, 4, 1, 0, 1, 1, 1, 1, 0, 3, 1, 0, 2, 1, 1, 0, 0, 0, 2,
       2, 1, 2, 2, 1, 1, 1, 1, 1, 4, 1, 0, 3, 1, 3, 1, 1, 3, 3, 4, 1, 2,
       4, 0, 2, 2, 1, 1, 4, 3, 2, 3, 1, 1, 3, 1, 1, 1, 4, 4, 3, 3, 0, 1,
       3, 1, 1, 3, 0, 2, 1, 1, 3, 4, 2, 4, 2, 1, 4, 0, 3, 2, 4, 1, 0, 4,
       0, 3, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 1, 0, 1, 2, 1, 3, 0, 3, 1, 1,
       2, 1, 1, 2, 2, 3, 2], dtype=int64)

### References 
#### The code was obtained by using tensorlfow

### https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier

### References using vancouver
#### 1. tf.contrib.learn.DNNClassifier  |  TensorFlow [Internet]. TensorFlow. [cited 4 December 2018]. Available from: https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier
